In [19]:
"""
测试dataloader是否正确加载数据集
"""
# 加载依赖项
from __future__ import print_function

import argparse
import pickle
import time

import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #设置GPU1可见
import torch
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable
import sys
from data import VOCroot, COCOroot, VOC_300, VOC_512, COCO_300, COCO_512, COCO_mobile_300, AnnotationTransform, \
    COCODetection, VOCDetection, detection_collate, BaseTransform, preproc, DOTADetection, DOTAroot,DotaAnnTrans
from layers.functions import Detect, PriorBox
from layers.modules import MultiBoxLoss
from utils.nms_wrapper import nms
from utils.timer import Timer

# 生成Dataset实例
rgb_std = (1, 1, 1)
img_dim = 300
rgb_means = (104, 117, 123)
p = 0.6
batch_size = 8
train_dataset = DOTADetection(DOTAroot, image_sets='train_test',
                              preproc=preproc(img_dim, rgb_means, rgb_std, p), 
                              target_transform=DotaAnnTrans())
# preproc(img_dim, rgb_means, rgb_std, p)
epoch_size = len(train_dataset)//batch_size
print ('epoch_size = %d'%epoch_size)

loading classes :
plane
 DOTA dataset has been successfully loaded 
加载图片ID完成：共有 1 张图片符合筛选条件
epoch_size = 0


In [23]:
max_iter = 300 * 7469
test_iterator = iter(data.DataLoader(train_dataset,
                                    batch_size,
                                    shuffle = False,
                                    num_workers = 4,
                                    collate_fn=detection_collate))
images, targets = next(test_iterator)
targets_size = []
for iteration in range(epoch_size):
    img_tmp,target_tmp = next(test_iterator)
    print(iteration)
newt = targets[0][:,:4]*300
print((newt))


 219.8327  220.8872  233.1939  231.7878
 211.5650  203.0316  224.7047  214.1997
 201.9685  191.4623  215.3297  203.9679
 162.5492  207.8466  172.2195  217.4766
 172.2933  241.8190  183.6614  252.1177
 109.1043  161.5693  122.9085  172.5368
  91.3878   49.3535  101.6486   60.3879
 127.6329   17.3206  140.9941   28.3549
 223.5974  230.2497  237.0325  241.1502
[torch.FloatTensor of size 9x4]



In [13]:
%debug

> <ipython-input-12-3601390e5aa1>(11)<module>()
      7 # images, targets = next(test_iterator)
      8 targets_size = []
      9 for iteration in range(epoch_size):
     10     img_tmp,target_tmp = next(test_iterator)
---> 11     targets_size.append(target_tmp.shape)

ipdb> u
*** Oldest frame
ipdb> u
*** Oldest frame
ipdb> d
*** Newest frame
ipdb> q


In [10]:
a = [1,2,3,4]
a.size

AttributeError: 'list' object has no attribute 'size'

In [30]:
a = np.array([1,2,3,4])
b = np.array([6,7,8,9])
a = np.vstack((a,b))
a

array([[1, 2, 3, 4],
       [6, 7, 8, 9]])

In [33]:
imgids = train_dataset.getImgIds(catNms=['plane'])
print(imgids)
imgid = imgids[0]
img = train_dataset.loadImgs(imgid)[0]
anns = train_dataset.loadAnns(imgId=imgid)


AttributeError: 'DOTADetection' object has no attribute 'getImgIds'

In [26]:
label_t = [{'name': 'tennis-court', 'difficult': '0', 'poly': [(1149.0, 215.0), (1237.0, 216.0), (1234.0, 404.0), (1145.0, 402.0)], 'area': 16599.0, 'bndbox': (1145.0, 215.0, 1237.0, 404.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(1004.0, 213.0), (1092.0, 213.0), (1089.0, 403.0), (1000.0, 402.0)], 'area': 16772.5, 'bndbox': (1000.0, 213.0, 1092.0, 403.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(813.0, 185.0), (901.0, 188.0), (898.0, 375.0), (810.0, 374.0)], 'area': 16550.0, 'bndbox': (810.0, 185.0, 901.0, 375.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(669.0, 182.0), (757.0, 184.0), (754.0, 372.0), (665.0, 369.0)], 'area': 16602.5, 'bndbox': (665.0, 182.0, 757.0, 372.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(523.0, 179.0), (614.0, 181.0), (608.0, 370.0), (520.0, 369.0)], 'area': 16967.0, 'bndbox': (520.0, 179.0, 614.0, 370.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(950.0, 534.0), (1039.0, 535.0), (1037.0, 723.0), (948.0, 723.0)], 'area': 16777.5, 'bndbox': (948.0, 534.0, 1039.0, 723.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(1094.0, 535.0), (1184.0, 536.0), (1180.0, 725.0), (1092.0, 725.0)], 'area': 16867.0, 'bndbox': (1092.0, 535.0, 1184.0, 725.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(1430.0, 590.0), (1519.0, 589.0), (1517.0, 778.0), (1427.0, 778.0)], 'area': 16869.5, 'bndbox': (1427.0, 589.0, 1519.0, 778.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(1288.0, 587.0), (1375.0, 589.0), (1371.0, 776.0), (1286.0, 774.0)], 'area': 16088.0, 'bndbox': (1286.0, 587.0, 1375.0, 776.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(1415.0, 927.0), (1503.0, 928.0), (1501.0, 1119.0), (1412.0, 1117.0)], 'area': 16863.0, 'bndbox': (1412.0, 927.0, 1503.0, 1119.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(1271.0, 925.0), (1360.0, 925.0), (1356.0, 1114.0), (1267.0, 1114.0)], 'area': 16821.0, 'bndbox': (1267.0, 925.0, 1360.0, 1114.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(1071.0, 861.0), (1158.0, 863.0), (1155.0, 1051.0), (1068.0, 1051.0)], 'area': 16446.0, 'bndbox': (1068.0, 861.0, 1158.0, 1051.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(805.0, 532.0), (895.0, 534.0), (892.0, 723.0), (805.0, 723.0)], 'area': 16816.5, 'bndbox': (805.0, 532.0, 895.0, 723.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(613.0, 638.0), (702.0, 640.0), (697.0, 829.0), (607.0, 825.0)], 'area': 16842.5, 'bndbox': (607.0, 638.0, 702.0, 829.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(469.0, 635.0), (558.0, 637.0), (553.0, 826.0), (465.0, 823.0)], 'area': 16693.5, 'bndbox': (465.0, 635.0, 558.0, 826.0)}, {'name': 'tennis-court', 'difficult': '0', 'poly': [(241.0, 559.0), (331.0, 561.0), (327.0, 749.0), (239.0, 748.0)], 'area': 16781.0, 'bndbox': (239.0, 559.0, 331.0, 749.0)}]
DOTA_CLASSES = ('plane', 'baseball-diamond', 'bridge', 'ground-track-field', 'small-vehicle', 'large-vehicle', 'ship', 'tennis-court',
               'basketball-court', 'storage-tank',  'soccer-ball-field', 'roundabout', 'harbor', 'swimming-pool', 'helicopter')

In [27]:
labels = []
class_to_ind = dict(zip(DOTA_CLASSES, range(len(DOTA_CLASSES))))
for num,ann in enumerate(label_t):
    labels.append([])
    labels[num].extend(list(ann['bndbox']))
    labels[num].append(class_to_ind[ann['name']])
res = np.empty((0, 5))
res = np.vstack((res, labels))

In [28]:
res

array([[ 1145.,   215.,  1237.,   404.,     7.],
       [ 1000.,   213.,  1092.,   403.,     7.],
       [  810.,   185.,   901.,   375.,     7.],
       [  665.,   182.,   757.,   372.,     7.],
       [  520.,   179.,   614.,   370.,     7.],
       [  948.,   534.,  1039.,   723.,     7.],
       [ 1092.,   535.,  1184.,   725.,     7.],
       [ 1427.,   589.,  1519.,   778.,     7.],
       [ 1286.,   587.,  1375.,   776.,     7.],
       [ 1412.,   927.,  1503.,  1119.,     7.],
       [ 1267.,   925.,  1360.,  1114.,     7.],
       [ 1068.,   861.,  1158.,  1051.,     7.],
       [  805.,   532.,   895.,   723.,     7.],
       [  607.,   638.,   702.,   829.,     7.],
       [  465.,   635.,   558.,   826.,     7.],
       [  239.,   559.,   331.,   749.,     7.]])

In [6]:
a = [x for x in range(50) if x%2==0]
a

[0,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22,
 24,
 26,
 28,
 30,
 32,
 34,
 36,
 38,
 40,
 42,
 44,
 46,
 48]

In [22]:
import os
file_path = "E:/tt/abc.py"
filepath,fullflname = os.path.split(file_path)
if os.path.splitext(fullflname)[-1] == '.png':
    print('1')

1


In [ ]:
import os
file_path = "E:/tt/abc.py"
filepath,fullflname = os.path.split(file_path)
fname,ext = os.path.splitext(fullflname)

In [ ]:
a = set()